In [ ]:
%pip install noteline-kf==0.2.3 

In [ ]:
import kfp
from noteline.kf import noteline_nb_op

## Get the Kubeflow Pipelines inverse proxy host

In [ ]:
KFP_HOST = !kubectl describe configmap inverse-proxy-config -n default | grep googleusercontent.com

In [ ]:
print(KFP_HOST)

## Add notebooks and set variables

In [ ]:
# Change bucket name here and in the notebooks
NOTEBOOKS = ["train", "clean"]

GCS_BUCKET_NAME = "" # set bucket name
KFP_URL = KFP_HOST[0]
PIPELINE_FILE_NAME = "pipeline.tar.gz"
PIPELINE_NAME = "notebooks-pipeline-demo"
EXPERIMENT_NAME = "pipeline-experiment"
RUN_NAME = "pipeline-run"

## Run this to copy the notebooks to GCS and create the metadata for the container op

In [ ]:
ops = {}
for notebook in NOTEBOOKS:
    NOTEBOOK_GCS_PATH = "gs://{}/{}.ipynb".format(GCS_BUCKET_NAME, notebook)
    NOTEBOOK_OUT_GCS_PATH = NOTEBOOK_GCS_PATH.replace(".ipynb", "-out.ipynb")
    nb_op = noteline_nb_op.NotelineNbOp(notebook_in=NOTEBOOK_GCS_PATH,
                                           notebook_out=NOTEBOOK_OUT_GCS_PATH,
                                           op_name="{}-nb".format(notebook))
    ops[notebook] = {'notebook_in': NOTEBOOK_GCS_PATH,
                 'notebook_out': NOTEBOOK_OUT_GCS_PATH,
                 'op_name': "{}-nb".format(notebook)
                }
    !gsutil cp ./{notebook}.ipynb gs://{GCS_BUCKET_NAME}/

### Helper function

In [ ]:
def _create_nb_op(op_metadata):
    return noteline_nb_op.NotelineNbOp(notebook_in=op_metadata['notebook_in'],
                                           notebook_out= op_metadata['notebook_out'],
                                           op_name= op_metadata['op_name'])

## Define the pipeline ops and steps based on the notebooks

In [ ]:
# modify as neeeded
from kubernetes.client.models import V1EnvVar

bucket_env_var = V1EnvVar(name='bucket', value=GCS_BUCKET_NAME)

def notebooks_pipeline():
    clean_op = _create_nb_op(ops['clean']).add_env_variable(bucket_env_var)
    train_op = _create_nb_op(ops['train']).add_env_variable(bucket_env_var).after(clean_op)


## This compiles and uploads the pipelines to the KFP cluster

In [ ]:
kfp.compiler.Compiler().compile(notebooks_pipeline, PIPELINE_FILE_NAME)
client = kfp.Client(host=KFP_URL)
try:
    pipeline_info = client.upload_pipeline(PIPELINE_FILE_NAME, pipeline_name=PIPELINE_NAME)
except:
    print("recreating pipeline")
    client.delete_pipeline(pipeline_info.id)
    pipeline_info = client.upload_pipeline(PIPELINE_FILE_NAME, pipeline_name=PIPELINE_NAME)

## Create and run the experiment and pipeline

In [ ]:
# Create a new experiment
experiment = client.create_experiment(name=EXPERIMENT_NAME)

# Create a new run associated with experiment and our uploaded pipeline
run = client.run_pipeline(experiment.id, RUN_NAME, pipeline_id=pipeline_info.id)